In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt  

In [ ]:
%pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
df = pd.read_csv('salary_data_cleaned.csv')

In [ ]:
df.head()

In [ ]:
## Simplify the job titles into different categories
def title_simplifier(title): 
    if 'data scientist' in title.lower(): 
        return 'Data Scientist'
    elif 'data engineer' in title.lower(): 
        return 'Data Engineer'
    elif 'analyst' in title.lower(): 
        return 'Analyst'
    elif 'machine learning' in title.lower(): 
        return 'Machine Learning Engineer'
    elif 'manager' in title.lower(): 
        return 'Manager'
    elif 'director' in title.lower(): 
        return 'Director' 
    else: 
        return 'NA' 

## Categorize based on seniority
def seniority(title): 
    if 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower() or 'sr.' in title.lower():
        return 'Senior' 
    elif 'jr' in title.lower() or 'junior' in title.lower() or 'jr.' in title.lower(): 
        return 'Junior' 
    else: 
        return 'NA'
    

In [ ]:
## Make a Job Simplified column in the DataFrame 
df['Job Simplified'] = df['Job Title'].apply(title_simplifier)
df['Job Simplified'].value_counts()

In [ ]:
## Make a Seniority column in the DataFrame 
df['Seniority'] = df['Job Title'].apply(seniority)
df['Seniority'].value_counts()

In [ ]:
## Adding Los Angeles value to CA
df.State = df['State'].apply(lambda x: x if x.lower() != 'los angeles' else 'CA')
df.State.value_counts()

In [ ]:
## Job Description Length 
df['Description Length'] = df['Job Description'].apply(lambda x: len(x))
df.head()

In [ ]:
df.columns

In [ ]:
## Competitor Count 
df['Competitor Count'] = df.Competitors.apply(lambda x: len(x.split(',')) if x!= '-1' else 0)
df.head()

In [ ]:
## Hourly Wage into Yearly Wage (multiply by 2000)
## Minimum Salary
df['Minimum Salary'] = df.apply(lambda row: 2*row['Minimum Salary'] if row.Hourly == 1 else row['Minimum Salary'], axis = 1)
## Maximum Salary 
df['Maximum Salary'] = df.apply(lambda row: 2*row['Maximum Salary'] if row.Hourly == 1 else row['Maximum Salary'], axis = 1)

In [ ]:
df[df['Hourly'] == 1][['Minimum Salary', 'Maximum Salary']]

In [ ]:
## Fix the new line character in the Company Name text
df.Company = df.Company.apply(lambda x: x.replace('\n', ''))
df.Company

In [ ]:
df.describe()

In [ ]:
## Histogram for the ratings of the company
df.Rating.hist()

In [ ]:
## Histogram for the age of the company
df['Company Age'].hist()

In [ ]:
## Histogram for the avg salary 
df['Average Salary'].hist()

In [ ]:
## Boxplot of Company Age and Average Salary 
df.boxplot(column = ['Company Age', 'Average Salary'])

In [ ]:
## Boxplot for Rating
df.boxplot(column = ['Rating'])

In [ ]:
## Correlations 
df[['Company Age', 'Average Salary', 'Rating', 'Description Length']].corr()

In [ ]:
## Correlation heatmap of the continuous variables 
cmap = sns.diverging_palette(220, 10, as_cmap = True)
sns.heatmap(df[['Company Age', 'Average Salary', 'Rating', 'Description Length', 'Competitor Count']].corr(), vmax = .3, center = 0, square = True, cmap = cmap, linewidths = .5, cbar_kws={"shrink": .5})

In [ ]:

df.columns

In [ ]:
## Categorical columns 
df_cat = df[['Location', 'Headquarters', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Company', 'State', 'Spark', 'AWS', 'Excel', 'Job Simplified',
       'Seniority']]

In [ ]:
for i in df_cat.columns: 
    cat = df_cat[i].value_counts()
    print("Graph for %s: Total = %d" % (i, len(cat)))
    chart = sns.barplot(x= cat, y = cat.index)
    plt.show()

In [ ]:
for i in df_cat[['Location', 'Headquarters', 'Company']].columns: 
    cat = df_cat[i].value_counts()[:20]
    print("Graph for %s: Total = %d" % (i, len(cat)))
    chart = sns.barplot(x= cat, y = cat.index)
    plt.show()

In [ ]:
## Average Salary by simplified Job titles 
pd.pivot_table(df, index = 'Job Simplified', values = 'Average Salary')

In [ ]:
## Average Salary by simplified job titles and seniority
pd.pivot_table(df, index = ['Job Simplified', 'Seniority'], values = 'Average Salary')

In [ ]:
## Average Salary by job locations ordered by highest average salary 
pd.pivot_table(df, index = 'State', values = 'Average Salary').sort_values('Average Salary', ascending = False)

In [ ]:
## Complete DataFrame 
pd.set_option('display.max_rows', None)

In [ ]:
## Average Salary by job locations, job title simplified ordered by highest average salary 
pd.pivot_table(df, index = ['State', 'Job Simplified'], values ='Average Salary').sort_values('State', ascending = False)

In [ ]:
## Average Salary of Data Scientists by State 
pd.pivot_table(df[df['Job Simplified'] == 'Data Scientist'], index ='State', values ='Average Salary').sort_values('Average Salary', ascending = False)

In [ ]:
## Pivot Tables of Average Salary vs. other columns 
df_pivots = df[['Industry', 'Sector', 'Revenue', 'Competitor Count', 'Hourly', 'Employer Provided', 'Python', 'R Studio', 'Spark',
                'AWS', 'Excel', 'Type of ownership', 'Average Salary']]

In [ ]:
for i in df_pivots.columns: 
    print(i)
    print(pd.pivot_table(df_pivots, index = i, values = 'Average Salary').sort_values('Average Salary', ascending = False))
    

In [ ]:
## Pivot table of Average Salary based on Revenue and Python
pd.pivot_table(df_pivots, index = 'Revenue', columns = 'Python', values = 'Average Salary', aggfunc = 'count').sort_values(by = [0,1], ascending = False)

In [ ]:
____## Word Cloud from the Job Description 
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import re

words = " ".join(df['Job Description'].dropna())

def clean_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    tokens = text.lower().split()
    tokens = [w for w in tokens if w not in STOPWORDS]
    return " ".join(tokens)

text = clean_text(words)

wc = WordCloud(background_color="white", max_words=2000, width=800, height=1500)
wc.generate(text)

plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()